## 3.15 Homework

### Dataset

In this homework, we will continue the New York City Airbnb Open Data. You can take it from
[Kaggle](https://www.kaggle.com/dgomonov/new-york-city-airbnb-open-data?select=AB_NYC_2019.csv)
or download from [here](https://raw.githubusercontent.com/alexeygrigorev/datasets/master/AB_NYC_2019.csv)
if you don't want to sign up to Kaggle.

We'll keep working with the `'price'` variable, and we'll transform it to a classification task.

### Features

For the rest of the homework, you'll need to use the features from the previous homework with additional two `'neighbourhood_group'` and `'room_type'`. So the whole feature set will be set as follows:

* `'neighbourhood_group'`,
* `'room_type'`,
* `'latitude'`,
* `'longitude'`,
* `'price'`,
* `'minimum_nights'`,
* `'number_of_reviews'`,
* `'reviews_per_month'`,
* `'calculated_host_listings_count'`,
* `'availability_365'`

Select only them and fill in the missing values with 0.

In [34]:
import pandas as pd
import numpy as np
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.metrics import mutual_info_score
from sklearn.linear_model import LogisticRegression, LinearRegression, Ridge
from sklearn.metrics import mean_squared_error

In [2]:
df = pd.read_csv('AB_NYC_2019.csv')
df.head()

,id,name,host_id,host_name,neighbourhood_group,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365
0,2539,Clean & quiet apt home by the park,2787,John,Brooklyn,Kensington,40.64749,-73.97237,Private room,149,1,9,2018-10-19,0.21,6,365
1,2595,Skylit Midtown Castle,2845,Jennifer,Manhattan,Midtown,40.75362,-73.98377,Entire home/apt,225,1,45,2019-05-21,0.38,2,355
2,3647,THE VILLAGE OF HARLEM....NEW YORK !,4632,Elisabeth,Manhattan,Harlem,40.80902,-73.94190,Private room,150,3,0,NaN,NaN,1,365
3,3831,Cozy Entire Floor of Brownstone,4869,LisaRoxanne,Brooklyn,Clinton Hill,40.68514,-73.95976,Entire home/apt,89,1,270,2019-07-05,4.64,1,194
4,5022,Entire Apt: Spacious Studio/Loft by central park,7192,Laura,Manhattan,East Harlem,40.79851,-73.94399,Entire home/apt,80,10,9,2018-11-19,0.10,1,0


In [3]:
columns = ['latitude', 'longitude', 'price', 'minimum_nights', 'number_of_reviews',
       'reviews_per_month', 'calculated_host_listings_count', 'availability_365', 'neighbourhood_group', 'room_type']
df = df[columns]

### Question 1

What is the most frequent observation (mode) for the column `'neighbourhood_group'`?

In [4]:
df.neighbourhood_group.mode()

0    Manhattan
dtype: object

In [5]:
df.neighbourhood_group.value_counts()

Manhattan        21661
Brooklyn         20104
Queens            5666
Bronx             1091
Staten Island      373
Name: neighbourhood_group, dtype: int64

### Split the data

* Split your data in train/val/test sets, with 60%/20%/20% distribution.
* Use Scikit-Learn for that (the `train_test_split` function) and set the seed to 42.
* Make sure that the target value ('price') is not in your dataframe.

In [6]:
X = df.drop(columns=['price'])
y = df.price

In [7]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.25, random_state=42)

### Question 2

* Create the [correlation matrix](https://www.google.com/search?q=correlation+matrix) for the numerical features of your train dataset.
   * In a correlation matrix, you compute the correlation coefficient between every pair of features in the dataset.
* What are the two features that have the biggest correlation in this dataset?

Example of a correlation matrix for the car price dataset:

<img src="images/correlation-matrix.png" />

In [8]:
X_train.corr()

,latitude,longitude,minimum_nights,number_of_reviews,reviews_per_month,calculated_host_listings_count,availability_365
latitude,1.000000,0.080301,0.027441,-0.006246,0.002646,0.019375,-0.005891
longitude,0.080301,1.000000,-0.060660,0.055084,0.140201,-0.117041,0.083666
minimum_nights,0.027441,-0.060660,1.000000,-0.076020,-0.111121,0.118647,0.138901
number_of_reviews,-0.006246,0.055084,-0.076020,1.000000,0.549792,-0.073167,0.174477
reviews_per_month,0.002646,0.140201,-0.111121,0.549792,1.000000,-0.007055,0.188765
calculated_host_listings_count,0.019375,-0.117041,0.118647,-0.073167,-0.007055,1.000000,0.225913
availability_365,-0.005891,0.083666,0.138901,0.174477,0.188765,0.225913,1.000000


### Make price binary

* We need to turn the price variable from numeric into binary.
* Let's create a variable `above_average` which is `1` if the price is above (or equal to) `152`.

In [9]:
above_average = (y_train >= 152).astype(int)

In [10]:
above_average

13575    0
48476    0
44499    0
17382    0
14638    0
        ..
13198    0
14583    0
6168     1
12248    0
20523    0
Name: price, Length: 29337, dtype: int32

### Question 3

* Calculate the mutual information score with the (binarized) price for the two categorical variables that we have. Use the training set only.
* Which of these two variables has bigger score?
* Round it to 2 decimal digits using `round(score, 2)`

In [11]:
df.dtypes == 'object'

latitude                          False
longitude                         False
price                             False
minimum_nights                    False
number_of_reviews                 False
reviews_per_month                 False
calculated_host_listings_count    False
availability_365                  False
neighbourhood_group                True
room_type                          True
dtype: bool

In [12]:
round(mutual_info_score(above_average, X_train.neighbourhood_group), 2)

0.05

In [13]:
round(mutual_info_score(above_average, X_train.room_type), 2)

0.14

### Question 4

* Now let's train a logistic regression
* Remember that we have two categorical variables in the data. Include them using one-hot encoding.
* Fit the model on the training dataset.
   * To make sure the results are reproducible across different versions of Scikit-Learn, fit the model with these parameters:
   * `model = LogisticRegression(solver='lbfgs', C=1.0, random_state=42)`
* Calculate the accuracy on the validation dataset and round it to 2 decimal digits.

In [14]:
X_train.isnull().sum()

latitude                             0
longitude                            0
minimum_nights                       0
number_of_reviews                    0
reviews_per_month                 6099
calculated_host_listings_count       0
availability_365                     0
neighbourhood_group                  0
room_type                            0
dtype: int64

In [15]:
reviews_per_month_mean = X_train.reviews_per_month.mean()
X_train = X_train.fillna(reviews_per_month_mean)
X_val = X_val.fillna(reviews_per_month_mean)
X_test = X_test.fillna(reviews_per_month_mean)

In [16]:
X_train_encoded = pd.get_dummies(X_train)
X_val_encoded = pd.get_dummies(X_val)
X_test_encoded = pd.get_dummies(X_test)

In [17]:
y_train_cls = (y_train >= 152).astype(int)
y_val_cls = (y_val >= 152).astype(int)
y_test_cls = (y_test >= 152).astype(int)

In [18]:
clf = LogisticRegression(solver='lbfgs', C=1.0, random_state=42)
clf.fit(X_train_encoded, y_train_cls)

C:\Users\dell\anaconda3\envs\GeoTechDaal\lib\site-packages\sklearn\linear_model\_logistic.py:765: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


LogisticRegression(random_state=42)

In [19]:
clf.score(X_val_encoded, y_val_cls)

0.7855608957971163

### Question 5

* We have 9 features: 7 numerical features and 2 categorical.
* Let's find the least useful one using the *feature elimination* technique.
* Train a model with all these features (using the same parameters as in Q4).
* Now exclude each feature from this set and train a model without it. Record the accuracy for each model.
* For each feature, calculate the difference between the original accuracy and the accuracy without the feature. 
* Which of following feature has the smallest difference? 
   * `neighbourhood_group`
   * `room_type` 
   * `number_of_reviews`
   * `reviews_per_month`

> **note**: the difference doesn't have to be positive

In [20]:
features = ['neighbourhood_group', 'room_type', 'number_of_reviews', 'reviews_per_month']

clf = LogisticRegression(solver='lbfgs', C=1.0, random_state=42)
clf.fit(pd.get_dummies(X_train[features]), y_train_cls)
score = clf.score(pd.get_dummies(X_val[features]), y_val_cls)

C:\Users\dell\anaconda3\envs\GeoTechDaal\lib\site-packages\sklearn\linear_model\_logistic.py:765: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


In [21]:
acc_diff = {}
for feature in features:
    train = X_train.drop(columns=feature)
    val = X_val.drop(columns=feature)
    
    clf = LogisticRegression(solver='lbfgs', C=1.0, random_state=42)
    clf.fit(pd.get_dummies(train), y_train_cls)
    acc_diff[feature] = score - clf.score(pd.get_dummies(val), y_val_cls)

C:\Users\dell\anaconda3\envs\GeoTechDaal\lib\site-packages\sklearn\linear_model\_logistic.py:765: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)
C:\Users\dell\anaconda3\envs\GeoTechDaal\lib\site-packages\sklearn\linear_model\_logistic.py:765: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/module

In [22]:
acc_diff

{'neighbourhood_group': 0.03200736271602411,
 'room_type': 0.06033336741998163,
 'number_of_reviews': -0.00603333674199813,
 'reviews_per_month': -0.004806217404642599}

### Question 6

* For this question, we'll see how to use a linear regression model from Scikit-Learn
* We'll need to use the original column `'price'`. Apply the logarithmic transformation to this column.
* Fit the Ridge regression model on the training data.
* This model has a parameter `alpha`. Let's try the following values: `[0, 0.01, 0.1, 1, 10]`
* Which of these alphas leads to the best RMSE on the validation set? Round your RMSE scores to 3 decimal digits.

If there are multiple options, select the smallest `alpha`.

In [42]:
for alpha in [0, 0.01, 0.1, 1, 10]:
    reg = Ridge(alpha= alpha)
    reg.fit(X_train_encoded, np.log1p(y_train))
    pred = reg.predict(X_val_encoded)
    rmse = np.sqrt(mean_squared_error(np.log1p(y_val), pred))
    print(alpha, "-->", round(rmse,3))

0 --> 0.497
0.01 --> 0.497
0.1 --> 0.497
1 --> 0.497
10 --> 0.498


C:\Users\dell\anaconda3\envs\GeoTechDaal\lib\site-packages\sklearn\linear_model\_ridge.py:148: LinAlgWarning: Ill-conditioned matrix (rcond=4.27462e-21): result may not be accurate.
  overwrite_a=True).T


## Submit the results

Submit your results here: https://forms.gle/xGpZhoq9Efm9E4RA9

It's possible that your answers won't match exactly. If it's the case, select the closest one.


## Deadline

The deadline for submitting is 27 September 2021, 17:00 CET. After that, the form will be closed.


## Nagivation

* [Machine Learning Zoomcamp course](../)
* [Session 3: Machine Learning for Classification](./)
* Previous: [Explore more](14-explore-more.md)